In [1]:
from data import *

Using TensorFlow backend.


# Data Augmentasi

Pembuatan data augmentasi menggunakan library **keras.preprocessing.image.ImageDataGenerator** yang bisa menggabungkan Deep Neural Network image dan label secara bersamaan.

## Tentukan parameter data generator
Gunakan parameter dengan metode random dictionary seperti dibawah ini.

In [2]:
data_gen_args = dict()#rotation_range = 0.1,
                    #width_shift_range = 0.01,
                    #height_shift_range = 0.01,
                    #shear_range = 0.05,
                    #zoom_range = 0.1,
                    #horizontal_flip = True,
                    #fill_mode = 'nearest')
myGenerator = trainGenerator(30, 'data/train', 'image', 'label', data_gen_args, save_to_dir = "data/train/aug")

## Membuat data augmentasi ke folder **'data/train/aug/'**

In [3]:
num_batch = 20
for i, batch in enumerate(myGenerator):
    print(i)
    if(i>=(num_batch-1)):
        break

Found 600 images belonging to 1 classes.
Found 600 images belonging to 1 classes.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


## Membuat .npy data (data augmentasi yang sudah dikompres)
Butuh memori tambahan untuk menyimpan data ini

In [4]:
image_arr,mask_arr = geneTrainNpy("data/train/aug/","data/train/aug/")
np.save("data/image_arr.npy",image_arr)
np.save("data/mask_arr.npy",mask_arr)

## Masking Test Data

In [33]:
import numpy as np 
import cv2

red = np.array([255, 0, 0])
green = np.array([0, 255, 0])
blue = np.array([0, 0, 255])
yellow = np.array([255, 255, 0])
aqua = np.array([0, 255, 255])

def masking(img, mask, color = None):
    mask_out = np.zeros((432, 532, 3), dtype = 'uint8')
    for i in range(mask.shape[0]-1):
        for j in range(mask.shape[1]-1):
            if (mask[i,j] >= 220):
                mask_out[i,j,:] = mask[i,j] 
                mask_out[i,j,:] = color
                cv2.cvtColor(img[i,j,:], COLOR_GRAY2BRG) = mask_out[i,j,:]
                img[i,j,:] = mask_out[i,j,:]
    segmented = img
    return segmented

In [34]:
import os
from os import listdir
from skimage import io

results_path = 'data/test/'
results_results = 'results/'
list_file = os.listdir(results_path)
#real = np.zeros((len(list_file)//2, 432, 532), dtype ='uint8')
masked = np.zeros((len(list_file)//2, 432, 532, 3), dtype ='uint8')

for i in range(len(list_file)//2):
    masked[i] = masking(io.imread(results_path+str(i)+'.png'), io.imread(results_path+str(i)+'_predict.png'), yellow)
    print('Creating... ' + results_path+str(i)+'_predict.png')
    io.imsave(os.path.join(results_results,"%d.png"%i), masked[i])

Creating... data/test/0_predict.png
Creating... data/test/1_predict.png
Creating... data/test/2_predict.png
Creating... data/test/3_predict.png
Creating... data/test/4_predict.png
Creating... data/test/5_predict.png
Creating... data/test/6_predict.png
Creating... data/test/7_predict.png
Creating... data/test/8_predict.png
Creating... data/test/9_predict.png
Creating... data/test/10_predict.png
Creating... data/test/11_predict.png
Creating... data/test/12_predict.png
Creating... data/test/13_predict.png
Creating... data/test/14_predict.png
Creating... data/test/15_predict.png
Creating... data/test/16_predict.png
Creating... data/test/17_predict.png
Creating... data/test/18_predict.png
Creating... data/test/19_predict.png
Creating... data/test/20_predict.png
Creating... data/test/21_predict.png
Creating... data/test/22_predict.png
Creating... data/test/23_predict.png
Creating... data/test/24_predict.png
Creating... data/test/25_predict.png
Creating... data/test/26_predict.png
Creating...

# CARA CONVERT PNG TO MP4
##ffmpeg -r 30 -f image2 -i %d.png -vcodec libx264 -crf 15  -pix_fmt yuv420p test.mp4

In [27]:
print(io.imread(results_path+str(1)+'.png').dtype)
print(io.imread(results_path+str(i)+'_predict.png').dtype)

uint8
uint8
